# Testing

In [1]:
from helpers import load_session, get_all_driver_features, merge_driver_features_with_session, load_or_build_profiles, build_circuit_profile_df

import pandas as pd
import numpy as np

In [2]:
YEAR          = 2024
EVENT_NAME    = "Spanish Grand Prix"     # exactly as FastF1 knows it
SESSION_LABEL = "FP2"

session_data  = load_session(YEAR, EVENT_NAME, SESSION_LABEL)

if session_data["status"] != "ok":      # FastF1 returned an error and no
    raise RuntimeError(session_data["reason"])  # fallback OpenF1 yet

session       = session_data["session"]  # FastF1.Session object

In [3]:
driver_df = get_all_driver_features(
    session,
    year=YEAR,
    session_name=SESSION_LABEL,
    throttle_ratio_min=0.40,
    throttle_ratio_max=0.85,
    braking_drop_kmh=30
)
print(driver_df.head(3))

⚠️ Tire degradation calc failed for SAR: Input y contains NaN.
⚠️ Tire degradation calc failed for GAS: Input y contains NaN.
⚠️ Tire degradation calc failed for PER: Input y contains NaN.
⚠️ Tire degradation calc failed for LEC: Input y contains NaN.
⚠️ Tire degradation calc failed for STR: Input y contains NaN.
⚠️ Tire degradation calc failed for MAG: Input y contains NaN.
⚠️ Tire degradation calc failed for TSU: Input y contains NaN.
⚠️ Tire degradation calc failed for OCO: Input y contains NaN.
⚠️ Tire degradation calc failed for HAM: Input y contains NaN.
           grand_prix   location driver     ratio compound  tyre_age  \
0  Spanish Grand Prix  Barcelona    VER  0.769295     SOFT       2.0   
1  Spanish Grand Prix  Barcelona    SAR  0.666209     SOFT       2.0   
2  Spanish Grand Prix  Barcelona    RIC  0.692146     SOFT       2.0   

   is_fresh_tyre  avg_rainfall  avg_track_temp  avg_air_temp  braking_events  \
0           True           0.0       43.185027     24.738324    

In [4]:
driver_df

,grand_prix,location,driver,ratio,compound,tyre_age,is_fresh_tyre,avg_rainfall,avg_track_temp,avg_air_temp,braking_events,session_uid,degradation_slope,brake_max_g,brake_avg_g,drs_activations,year,session,event
0,Spanish Grand Prix,Barcelona,VER,0.769295,SOFT,2.0,True,0.0,43.185027,24.738324,2,2024_Barcelona_FP2,3.590869,5.663156,3.590528,3,2024,FP2,Spanish Grand Prix
1,Spanish Grand Prix,Barcelona,SAR,0.666209,SOFT,2.0,True,0.0,43.038062,24.636505,5,2024_Barcelona_FP2,NaN,7.362102,4.532918,3,2024,FP2,Spanish Grand Prix
2,Spanish Grand Prix,Barcelona,RIC,0.692146,SOFT,2.0,True,0.0,43.062762,24.800000,4,2024_Barcelona_FP2,7.683500,6.017103,4.193601,3,2024,FP2,Spanish Grand Prix
3,Spanish Grand Prix,Barcelona,NOR,0.703094,MEDIUM,2.0,True,0.0,43.007407,24.618695,4,2024_Barcelona_FP2,5.302314,4.866774,3.386875,3,2024,FP2,Spanish Grand Prix
4,Spanish Grand Prix,Barcelona,GAS,0.750702,SOFT,2.0,True,0.0,42.675133,24.825044,2,2024_Barcelona_FP2,NaN,5.167629,3.556398,3,2024,FP2,Spanish Grand Prix
5,Spanish Grand Prix,Barcelona,PER,0.750576,SOFT,2.0,True,0.0,42.369286,24.700000,4,2024_Barcelona_FP2,NaN,5.840129,4.337539,3,2024,FP2,Spanish Grand Prix
6,Spanish Grand Prix,Barcelona,ALO,0.715578,SOFT,2.0,True,0.0,43.477202,24.853886,2,2024_Barcelona_FP2,11.886000,6.017103,4.647102,3,2024,FP2,Spanish Grand Prix
7,Spanish Grand Prix,Barcelona,LEC,0.688622,SOFT,2.0,True,0.0,43.000000,24.627160,4,2024_Barcelona_FP2,NaN,5.946313,3.918227,3,2024,FP2,Spanish Grand Prix
8,Spanish Grand Prix,Barcelona,STR,0.764910,SOFT,2.0,True,0.0,43.300000,24.700000,3,2024_Barcelona_FP2,NaN,5.486182,4.090925,3,2024,FP2,Spanish Grand Prix
9,Spanish Grand Prix,Barcelona,MAG,0.687754,SOFT,2.0,True,0.0,43.204813,24.668271,3,2024_Barcelona_FP2,NaN,6.795787,4.791695,3,2024,FP2,Spanish Grand Prix


In [5]:
df_profiles, df_skipped = load_or_build_profiles(
    cache_path="data/circuit_profiles.csv",
    start_year=2024,
    end_year=2025
)

✅ Using cached circuit profile file.


In [7]:
df_profiles.loc[df_profiles.event=='Spanish Grand Prix']

,year,event,location,session,real_altitude,lap_length,telemetry_source,num_drs_zones,drs_total_len_m,drs_pct_of_lap,avg_speed,top_speed,braking_events,low_pct,med_pct,high_pct,slow_corners,medium_corners,fast_corners,chicanes,air_temp_avg,track_temp_avg,rain_detected
45,2024,Spanish Grand Prix,Montmeló,FP1,121.0,4635.323889,fastf1,2,3455.285000,0.745425,227.060284,322.0,8,0.039007,0.297872,0.663121,0,4,2,1,28.165432,47.092593,False
46,2024,Spanish Grand Prix,Montmeló,FP2,121.0,4619.725833,fastf1,2,3404.260278,0.736897,228.704626,320.0,9,0.039146,0.291815,0.669039,0,5,3,1,24.865432,43.051852,False
47,2024,Spanish Grand Prix,Montmeló,FP3,121.0,4635.788611,fastf1,2,3410.983333,0.735794,228.214815,324.0,8,0.040741,0.311111,0.648148,0,6,1,0,26.441250,43.355000,False
48,2024,Spanish Grand Prix,Montmeló,Q,121.0,4633.510833,fastf1,2,3379.590278,0.729380,233.659259,325.0,6,0.014815,0.296296,0.688889,0,6,3,0,24.089474,36.535526,False
49,2024,Spanish Grand Prix,Montmeló,R,121.0,4612.067500,fastf1,0,0.000000,NaN,216.527397,327.0,5,0.065068,0.335616,0.599315,1,4,3,1,24.132468,41.096104,True
160,2025,Spanish Grand Prix,Montmeló,FP1,121.0,4621.017778,fastf1,2,3425.459167,0.741278,228.416357,326.0,5,0.048327,0.286245,0.665428,0,3,3,1,30.375000,48.778750,False
161,2025,Spanish Grand Prix,Montmeló,FP2,121.0,4615.722222,fastf1,2,3458.291667,0.749242,229.549451,328.0,6,0.051282,0.282051,0.666667,0,4,0,0,30.128049,45.769512,False
162,2025,Spanish Grand Prix,Montmeló,FP3,121.0,4620.372222,fastf1,2,3442.645833,0.745101,230.645283,326.0,9,0.033962,0.290566,0.675472,0,2,1,0,29.920408,46.824490,False
163,2025,Spanish Grand Prix,Montmeló,Q,121.0,4628.872778,fastf1,2,3412.334444,0.737185,233.095941,328.0,6,0.022140,0.291513,0.686347,0,5,3,1,29.136364,46.823377,False
164,2025,Spanish Grand Prix,Montmeló,R,121.0,4616.068333,fastf1,1,4608.310000,0.998319,221.088652,309.0,7,0.046099,0.329787,0.624113,0,5,3,1,29.120000,48.543226,False


In [8]:
merge_driver_features_with_session(driver_df, df_profiles)

,grand_prix,location_x,driver,ratio,compound,tyre_age,is_fresh_tyre,avg_rainfall,avg_track_temp,avg_air_temp,braking_events_x,session_uid,degradation_slope,brake_max_g,brake_avg_g,drs_activations,year,session,event,location_y,real_altitude,lap_length,telemetry_source,num_drs_zones,drs_total_len_m,drs_pct_of_lap,avg_speed,top_speed,braking_events_y,low_pct,med_pct,high_pct,slow_corners,medium_corners,fast_corners,chicanes,air_temp_avg,track_temp_avg,rain_detected
0,Spanish Grand Prix,Barcelona,VER,0.769295,SOFT,2.0,True,0.0,43.185027,24.738324,2,2024_Barcelona_FP2,3.590869,5.663156,3.590528,3,2024,FP2,Spanish Grand Prix,Montmeló,121.0,4619.725833,fastf1,2,3404.260278,0.736897,228.704626,320.0,9,0.039146,0.291815,0.669039,0,5,3,1,24.865432,43.051852,False
1,Spanish Grand Prix,Barcelona,SAR,0.666209,SOFT,2.0,True,0.0,43.038062,24.636505,5,2024_Barcelona_FP2,NaN,7.362102,4.532918,3,2024,FP2,Spanish Grand Prix,Montmeló,121.0,4619.725833,fastf1,2,3404.260278,0.736897,228.704626,320.0,9,0.039146,0.291815,0.669039,0,5,3,1,24.865432,43.051852,False
2,Spanish Grand Prix,Barcelona,RIC,0.692146,SOFT,2.0,True,0.0,43.062762,24.800000,4,2024_Barcelona_FP2,7.683500,6.017103,4.193601,3,2024,FP2,Spanish Grand Prix,Montmeló,121.0,4619.725833,fastf1,2,3404.260278,0.736897,228.704626,320.0,9,0.039146,0.291815,0.669039,0,5,3,1,24.865432,43.051852,False
3,Spanish Grand Prix,Barcelona,NOR,0.703094,MEDIUM,2.0,True,0.0,43.007407,24.618695,4,2024_Barcelona_FP2,5.302314,4.866774,3.386875,3,2024,FP2,Spanish Grand Prix,Montmeló,121.0,4619.725833,fastf1,2,3404.260278,0.736897,228.704626,320.0,9,0.039146,0.291815,0.669039,0,5,3,1,24.865432,43.051852,False
4,Spanish Grand Prix,Barcelona,GAS,0.750702,SOFT,2.0,True,0.0,42.675133,24.825044,2,2024_Barcelona_FP2,NaN,5.167629,3.556398,3,2024,FP2,Spanish Grand Prix,Montmeló,121.0,4619.725833,fastf1,2,3404.260278,0.736897,228.704626,320.0,9,0.039146,0.291815,0.669039,0,5,3,1,24.865432,43.051852,False
5,Spanish Grand Prix,Barcelona,PER,0.750576,SOFT,2.0,True,0.0,42.369286,24.700000,4,2024_Barcelona_FP2,NaN,5.840129,4.337539,3,2024,FP2,Spanish Grand Prix,Montmeló,121.0,4619.725833,fastf1,2,3404.260278,0.736897,228.704626,320.0,9,0.039146,0.291815,0.669039,0,5,3,1,24.865432,43.051852,False
6,Spanish Grand Prix,Barcelona,ALO,0.715578,SOFT,2.0,True,0.0,43.477202,24.853886,2,2024_Barcelona_FP2,11.886000,6.017103,4.647102,3,2024,FP2,Spanish Grand Prix,Montmeló,121.0,4619.725833,fastf1,2,3404.260278,0.736897,228.704626,320.0,9,0.039146,0.291815,0.669039,0,5,3,1,24.865432,43.051852,False
7,Spanish Grand Prix,Barcelona,LEC,0.688622,SOFT,2.0,True,0.0,43.000000,24.627160,4,2024_Barcelona_FP2,NaN,5.946313,3.918227,3,2024,FP2,Spanish Grand Prix,Montmeló,121.0,4619.725833,fastf1,2,3404.260278,0.736897,228.704626,320.0,9,0.039146,0.291815,0.669039,0,5,3,1,24.865432,43.051852,False
8,Spanish Grand Prix,Barcelona,STR,0.764910,SOFT,2.0,True,0.0,43.300000,24.700000,3,2024_Barcelona_FP2,NaN,5.486182,4.090925,3,2024,FP2,Spanish Grand Prix,Montmeló,121.0,4619.725833,fastf1,2,3404.260278,0.736897,228.704626,320.0,9,0.039146,0.291815,0.669039,0,5,3,1,24.865432,43.051852,False
9,Spanish Grand Prix,Barcelona,MAG,0.687754,SOFT,2.0,True,0.0,43.204813,24.668271,3,2024_Barcelona_FP2,NaN,6.795787,4.791695,3,2024,FP2,Spanish Grand Prix,Montmeló,121.0,4619.725833,fastf1,2,3404.260278,0.736897,228.704626,320.0,9,0.039146,0.291815,0.669039,0,5,3,1,24.865432,43.051852,False


# Work in progress

1. quali pace
2. race pace
3. Pirelli tire info
4. when to query the data?
    - the time of the race is not consistent
    - on sprint race weekends the quali takes place on Friday
    - US/Asia/Australia races happen at different times

In [ ]:
def create_pirelli_tyre_info(location, traction, evolution, lateral_load, abrasion, braking, grip, tyre_stress, downforce):
    return pd.DataFrame([{
        'Location': location,
        'traction': traction,
        'evolution': evolution,
        'lateral_load': lateral_load,
        'abrasion': abrasion,
        'braking': braking,
        'grip': grip,
        'tyre_stress': tyre_stress,
        'downforce':downforce
        }])


In [ ]:
bahrain_pirelli = create_pirelli_tyre_info('Sakhir', 4, 4, 3, 5, 4, 3, 3, 3)
jeddah_pirelli = create_pirelli_tyre_info('Jeddah', 2, 3, 4, 2, 2, 3, 3, 2)
melbourne_pirelli = create_pirelli_tyre_info('Melbourne', 2, 4, 3, 2, 2, 3, 3, 3)
baku_pirelli = create_pirelli_tyre_info('Baku', 5, 5, 1, 1, 4, 1, 3, 1)
miami_pirelli = create_pirelli_tyre_info('Miami',3,5,3,2,3,3,3,2)
monaco_pirelli = create_pirelli_tyre_info('Monaco', 5,5,1,1,2,1,1,5)
barcelona_pirelli = create_pirelli_tyre_info('Barcelona', 3,3,5,4,3,3,5,4) #2023
barcelona_pirelli = create_pirelli_tyre_info('Barcelona', 3,3,4,4,3,3,4,4) #2022
montreal_pirelli = create_pirelli_tyre_info('Montreal', 5,5,1,2,5,1,3,1)

spielberg_pirelli = create_pirelli_tyre_info('Spielberg', 2,3,5)
silverstone_pirelli = create_pirelli_tyre_info('Silverstone', 4,4,10)
budapest_pirelli = create_pirelli_tyre_info('Budapest', 3,7,4)
spa_pirelli = create_pirelli_tyre_info('Spa', 3,8,8)
imola_pirelli = create_pirelli_tyre_info('Imola', 5,3,11)
abu_dhabi_pirelli = create_pirelli_tyre_info('Abu Dhabi',3,6,7)
las_vegas_pirelli = create_pirelli_tyre_info('Las Vegas', 7,4,6)
mexico_pirelli = create_pirelli_tyre_info('Mexico City', 7,6,4)
sao_paulo_pirelli = create_pirelli_tyre_info('Sao Paulo', 1,6,8)
suzuka_pirelli = create_pirelli_tyre_info('Suzuka', 3,4,11)
qatar_pirelli = create_pirelli_tyre_info('Qatar', 1,4,11)
austin_pirelli = create_pirelli_tyre_info('Austin', 7,5,8)
zandvoort_pirelli = create_pirelli_tyre_info('Zandvoort', 2,6,6)
monza_pirelli = create_pirelli_tyre_info('Monza', 2,5,4)
singapore_pirelli = create_pirelli_tyre_info('Singapore', 10,8,5) #2022
singapore_pirelli = create_pirelli_tyre_info('Singapore', 7,7,5) #2023


In [ ]:
pirelli = pd.concat([
    bahrain_pirelli,
    jeddah_pirelli,
    melbourne_pirelli,
    baku_pirelli,
    miami_pirelli ,
    monaco_pirelli ,
    barcelona_pirelli,
    montreal_pirelli ,
    ], 
    ignore_index=True
    )